# Building with AutoGen 
### Please Complete the **[environment setup](part1_environment_setup.ipynb)** first before progressing with the notebook!


## 1.Starting Building with AutoGen 
Let's start with a simple example of how AutoGen is integrated Azure OpenAI. 
The example will confirm Azure OpenAI endpoints and keys are correctly configured and that AutoGen is installed and working correctly.


In [4]:
import os 
import asyncio
from dotenv import load_dotenv
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console


# Load environment variables from .env
load_dotenv()

# Create a minimal Azure OpenAI client.
azure_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4o-mini",
    model="gpt-4o-mini",
    api_version="2024-06-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

# Create an assistant agent with a simple system message.
agent = AssistantAgent(
    name="azure_test_agent",
    model_client=azure_client,
    system_message="You are a helpful Azure test assistant."
)

async def main():
    # A quick test question to make sure the connection works.
    user_input = "Hello! Are you working?"
    await Console(agent.run_stream(task=user_input))

if __name__ == "__main__":
    await main()


---------- user ----------
Hello! Are you working?
---------- azure_test_agent ----------
Hello! Yes, I'm here and ready to help you. How can I assist you today?


The **autogen** framework provides multiple agent roles—such as `AssistantAgent`, `UserAgent`, `FunctionAgent` (for function or tool calls), and `MultiAgent` (for orchestrating multiple participants)—each designed for specialized behaviors in a conversation. For example:

- `AssistantAgent`: Represents a helpful AI assistant.
- `FunctionAgent`: Handles specific tools or API calls.
- `MultiAgent`: Coordinates interactions among different agents.

This modular design makes it straightforward to build and manage multi-turn, multi-role dialogues, assigning clear, distinct responsibilities to each agent.


`AssistantAgent` we used here is a high-level wrapper around a language model client (such as an AzureOpenAI model) that manages:

1. **Conversational Context**: It keeps track of all previous exchanges and organizes them into a coherent dialog.
2. **System or Instruction Prompt**: You can give it a “system message” that defines the overarching instructions or persona, such as:
   > "You are a helpful Azure test assistant."

By consolidating these elements, `AssistantAgent` simplifies the process of building chat interfaces on top of LLMs, handling how user messages and model responses flow, and ensuring all conversation logic is neatly contained in a single agent object.
